# ElemNet training

# 1. SSUB

In [1]:
#https://github.com/wolverton-research-group/qmpy/blob/master/qmpy/data/thermodata/ssub.dat
from jarvis.db.figshare import  get_request_data
dat = get_request_data(js_tag="ssub.json",url="https://figshare.com/ndownloader/files/40084921")


Loading the zipfile...
Loading completed.


In [2]:
import pandas as pd
df = pd.DataFrame(dat)

In [3]:
df

,id,formula_energy,formula
0,465,-2.841796e+00,CaO4W
1,1197,1.766051e-09,Ho
2,1759,-3.555857e-01,PbTe
3,1454,-9.540103e-01,MoS2
4,218,-6.564038e-02,BeH2
...,...,...,...
1722,1482,-1.892842e+00,NZr
1723,773,-2.404511e+00,CrLiO2
1724,251,-4.946426e-01,BiNa3
1725,328,-7.213532e-01,Br4Ge


In [4]:
import json,zipfile
import numpy as np
path = "../../benchmarks/AI/SinglePropertyPrediction/ssub_formula_energy.json.zip"
js_tag = "ssub_formula_energy.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))

In [5]:
id_data.keys()

dict_keys(['train', 'test'])

In [6]:
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')

In [7]:
print(len(train_ids),len(test_ids),len(train_ids)+len(test_ids))

1381 345 1726


In [8]:
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

In [9]:
len(train_df),  len(test_df)

(1381, 345)

In [10]:
%%time
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

lgbm = lgb.LGBMRegressor(
    # device="gpu",
    n_estimators=1170,
    learning_rate=0.15,
    num_leaves=273,
)
X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['formula_energy'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['formula_energy'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')


CPU times: user 235 ms, sys: 26.4 ms, total: 261 ms
Wall time: 326 ms


In [11]:
#Split train into train-val split
n_train = int(len(X_train)*0.8)
n_val = len(X_train)-n_train
n_train,n_val,X_train.shape
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_val = X_train[-n_val:]
y_val = y_train[-n_val:]

In [12]:
X_train.shape,X_test.shape

((1104, 103), (345, 103))

In [13]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error

2023-05-21 08:28:49.298895: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-21 08:28:49.302652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-21 08:28:49.302663: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [14]:
#Network from "ElemNet: Deep Learning the Chemistry of Materials From Only Elemental Composition"
#Link to paper: https://www.nature.com/articles/s41598-018-35934-y
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = Activation('relu')(layer_1)

layer_2 = Dense(1024)(layer_1)
layer_2 = Activation('relu')(layer_2)

layer_3 = Dense(1024)(layer_2)
layer_3 = Activation('relu')(layer_3)

layer_4 = Dense(1024)(layer_3)
layer_4 = Activation('relu')(layer_4)
layer_4 = Dropout(0.2)(layer_4, training=True)

layer_5 = Dense(512)(layer_4)
layer_5 = Activation('relu')(layer_5)

layer_6 = Dense(512)(layer_5)
layer_6 = Activation('relu')(layer_6)

layer_7 = Dense(512)(layer_6)
layer_7 = Activation('relu')(layer_7)
layer_7 = Dropout(0.1)(layer_7, training=True)

layer_8 = Dense(256)(layer_7)
layer_8 = Activation('relu')(layer_8)

layer_9 = Dense(256)(layer_8)
layer_9 = Activation('relu')(layer_9)

layer_10 = Dense(256)(layer_9)
layer_10 = Activation('relu')(layer_10)
layer_10 = Dropout(0.3)(layer_10, training=True)

layer_11 = Dense(128)(layer_10)
layer_11 = Activation('relu')(layer_11)

layer_12 = Dense(128)(layer_11)
layer_12 = Activation('relu')(layer_12)

layer_13 = Dense(128)(layer_12)
layer_13 = Activation('relu')(layer_13)
layer_13 = Dropout(0.2)(layer_13, training=True)

layer_14 = Dense(64)(layer_13)
layer_14 = Activation('relu')(layer_14)

layer_15 = Dense(64)(layer_14)
layer_15 = Activation('relu')(layer_15)

layer_16 = Dense(32)(layer_15)
layer_16 = Activation('relu')(layer_16)

out_layer = Dense(1)(layer_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


2023-05-21 08:28:53.014764: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-21 08:28:53.019073: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-21 08:28:53.019150: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-21 08:28:53.019198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-21 08:28:53.019240: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [15]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000
35/35 - 2s - loss: 1.3042 - mean_absolute_error: 1.3042 - val_loss: 0.9015 - val_mean_absolute_error: 0.9015 - 2s/epoch - 57ms/step
Epoch 2/3000
35/35 - 1s - loss: 0.7729 - mean_absolute_error: 0.7729 - val_loss: 0.5854 - val_mean_absolute_error: 0.5854 - 828ms/epoch - 24ms/step
Epoch 3/3000
35/35 - 1s - loss: 0.5416 - mean_absolute_error: 0.5416 - val_loss: 0.4768 - val_mean_absolute_error: 0.4768 - 763ms/epoch - 22ms/step
Epoch 4/3000
35/35 - 1s - loss: 0.5146 - mean_absolute_error: 0.5146 - val_loss: 0.4984 - val_mean_absolute_error: 0.4984 - 804ms/epoch - 23ms/step
Epoch 5/3000
35/35 - 1s - loss: 0.4581 - mean_absolute_error: 0.4581 - val_loss: 0.3856 - val_mean_absolute_error: 0.3856 - 826ms/epoch - 24ms/step
Epoch 6/3000
35/35 - 1s - loss: 0.4120 - mean_absolute_error: 0.4120 - val_loss: 0.3266 - val_mean_absolute_error: 0.3266 - 807ms/epoch - 23ms/step
Epoch 7/3000
35/35 - 1s - loss: 0.3778 - mean_absolute_error: 0.3778 - val_loss: 0.3127 - val_mean_absolute_error: 

35/35 - 1s - loss: 0.1176 - mean_absolute_error: 0.1176 - val_loss: 0.0915 - val_mean_absolute_error: 0.0915 - 793ms/epoch - 23ms/step
Epoch 57/3000
35/35 - 1s - loss: 0.1261 - mean_absolute_error: 0.1261 - val_loss: 0.0933 - val_mean_absolute_error: 0.0933 - 783ms/epoch - 22ms/step
Epoch 58/3000
35/35 - 1s - loss: 0.1179 - mean_absolute_error: 0.1179 - val_loss: 0.1125 - val_mean_absolute_error: 0.1125 - 740ms/epoch - 21ms/step
Epoch 59/3000
35/35 - 1s - loss: 0.1236 - mean_absolute_error: 0.1236 - val_loss: 0.1099 - val_mean_absolute_error: 0.1099 - 779ms/epoch - 22ms/step
Epoch 60/3000
35/35 - 1s - loss: 0.1165 - mean_absolute_error: 0.1165 - val_loss: 0.0915 - val_mean_absolute_error: 0.0915 - 809ms/epoch - 23ms/step
Epoch 61/3000
35/35 - 1s - loss: 0.1119 - mean_absolute_error: 0.1119 - val_loss: 0.1359 - val_mean_absolute_error: 0.1359 - 804ms/epoch - 23ms/step
Epoch 62/3000
35/35 - 1s - loss: 0.1208 - mean_absolute_error: 0.1208 - val_loss: 0.1165 - val_mean_absolute_error: 0.11

35/35 - 1s - loss: 0.0960 - mean_absolute_error: 0.0960 - val_loss: 0.0807 - val_mean_absolute_error: 0.0807 - 792ms/epoch - 23ms/step
Epoch 112/3000
35/35 - 1s - loss: 0.1046 - mean_absolute_error: 0.1046 - val_loss: 0.0729 - val_mean_absolute_error: 0.0729 - 792ms/epoch - 23ms/step
Epoch 113/3000
35/35 - 1s - loss: 0.0928 - mean_absolute_error: 0.0928 - val_loss: 0.0828 - val_mean_absolute_error: 0.0828 - 809ms/epoch - 23ms/step
Epoch 114/3000
35/35 - 1s - loss: 0.0926 - mean_absolute_error: 0.0926 - val_loss: 0.0807 - val_mean_absolute_error: 0.0807 - 802ms/epoch - 23ms/step
Epoch 115/3000
35/35 - 1s - loss: 0.0942 - mean_absolute_error: 0.0942 - val_loss: 0.0995 - val_mean_absolute_error: 0.0995 - 810ms/epoch - 23ms/step
Epoch 116/3000
35/35 - 1s - loss: 0.0952 - mean_absolute_error: 0.0952 - val_loss: 0.1197 - val_mean_absolute_error: 0.1197 - 793ms/epoch - 23ms/step
Epoch 117/3000
35/35 - 1s - loss: 0.0983 - mean_absolute_error: 0.0983 - val_loss: 0.0949 - val_mean_absolute_error

Epoch 166/3000
35/35 - 1s - loss: 0.0825 - mean_absolute_error: 0.0825 - val_loss: 0.0694 - val_mean_absolute_error: 0.0694 - 795ms/epoch - 23ms/step
Epoch 167/3000
35/35 - 1s - loss: 0.0706 - mean_absolute_error: 0.0706 - val_loss: 0.0706 - val_mean_absolute_error: 0.0706 - 829ms/epoch - 24ms/step
Epoch 168/3000
35/35 - 1s - loss: 0.0762 - mean_absolute_error: 0.0762 - val_loss: 0.0748 - val_mean_absolute_error: 0.0748 - 827ms/epoch - 24ms/step
Epoch 169/3000
35/35 - 1s - loss: 0.0788 - mean_absolute_error: 0.0788 - val_loss: 0.0569 - val_mean_absolute_error: 0.0569 - 779ms/epoch - 22ms/step
Epoch 170/3000
35/35 - 1s - loss: 0.0801 - mean_absolute_error: 0.0801 - val_loss: 0.0672 - val_mean_absolute_error: 0.0672 - 781ms/epoch - 22ms/step
Epoch 171/3000
35/35 - 1s - loss: 0.0861 - mean_absolute_error: 0.0861 - val_loss: 0.1022 - val_mean_absolute_error: 0.1022 - 768ms/epoch - 22ms/step
Epoch 172/3000
35/35 - 1s - loss: 0.0897 - mean_absolute_error: 0.0897 - val_loss: 0.0651 - val_mean

Epoch 221/3000
35/35 - 1s - loss: 0.0770 - mean_absolute_error: 0.0770 - val_loss: 0.0555 - val_mean_absolute_error: 0.0555 - 791ms/epoch - 23ms/step
Epoch 222/3000
35/35 - 1s - loss: 0.0755 - mean_absolute_error: 0.0755 - val_loss: 0.0693 - val_mean_absolute_error: 0.0693 - 785ms/epoch - 22ms/step
Epoch 223/3000
35/35 - 1s - loss: 0.0683 - mean_absolute_error: 0.0683 - val_loss: 0.0554 - val_mean_absolute_error: 0.0554 - 762ms/epoch - 22ms/step
Epoch 224/3000
35/35 - 1s - loss: 0.0666 - mean_absolute_error: 0.0666 - val_loss: 0.0573 - val_mean_absolute_error: 0.0573 - 789ms/epoch - 23ms/step
Epoch 225/3000
35/35 - 1s - loss: 0.0669 - mean_absolute_error: 0.0669 - val_loss: 0.0792 - val_mean_absolute_error: 0.0792 - 818ms/epoch - 23ms/step
Epoch 226/3000
35/35 - 1s - loss: 0.0745 - mean_absolute_error: 0.0745 - val_loss: 0.0594 - val_mean_absolute_error: 0.0594 - 781ms/epoch - 22ms/step
Epoch 227/3000
35/35 - 1s - loss: 0.0689 - mean_absolute_error: 0.0689 - val_loss: 0.0624 - val_mean

...

Epoch 496/3000
35/35 - 1s - loss: 0.0480 - mean_absolute_error: 0.0480 - val_loss: 0.0403 - val_mean_absolute_error: 0.0403 - 829ms/epoch - 24ms/step
Epoch 497/3000
35/35 - 1s - loss: 0.0482 - mean_absolute_error: 0.0482 - val_loss: 0.0462 - val_mean_absolute_error: 0.0462 - 837ms/epoch - 24ms/step
Epoch 498/3000
35/35 - 1s - loss: 0.0475 - mean_absolute_error: 0.0475 - val_loss: 0.0393 - val_mean_absolute_error: 0.0393 - 774ms/epoch - 22ms/step
Epoch 499/3000
35/35 - 1s - loss: 0.0427 - mean_absolute_error: 0.0427 - val_loss: 0.0400 - val_mean_absolute_error: 0.0400 - 775ms/epoch - 22ms/step
Epoch 500/3000
35/35 - 1s - loss: 0.0433 - mean_absolute_error: 0.0433 - val_loss: 0.0411 - val_mean_absolute_error: 0.0411 - 798ms/epoch - 23ms/step
Epoch 501/3000
35/35 - 1s - loss: 0.0416 - mean_absolute_error: 0.0416 - val_loss: 0.0356 - val_mean_absolute_error: 0.0356 - 809ms/epoch - 23ms/step
Epoch 502/3000
35/35 - 1s - loss: 0.0444 - mean_absolute_error: 0.0444 - val_loss: 0.0402 - val_mean

Epoch 551/3000
35/35 - 1s - loss: 0.0511 - mean_absolute_error: 0.0511 - val_loss: 0.0447 - val_mean_absolute_error: 0.0447 - 831ms/epoch - 24ms/step
Epoch 552/3000
35/35 - 1s - loss: 0.0393 - mean_absolute_error: 0.0393 - val_loss: 0.0312 - val_mean_absolute_error: 0.0312 - 840ms/epoch - 24ms/step
Epoch 553/3000
35/35 - 1s - loss: 0.0412 - mean_absolute_error: 0.0412 - val_loss: 0.0366 - val_mean_absolute_error: 0.0366 - 825ms/epoch - 24ms/step
Epoch 554/3000
35/35 - 1s - loss: 0.0409 - mean_absolute_error: 0.0409 - val_loss: 0.0361 - val_mean_absolute_error: 0.0361 - 825ms/epoch - 24ms/step
Epoch 555/3000
35/35 - 1s - loss: 0.0479 - mean_absolute_error: 0.0479 - val_loss: 0.0354 - val_mean_absolute_error: 0.0354 - 835ms/epoch - 24ms/step
Epoch 556/3000
35/35 - 1s - loss: 0.0388 - mean_absolute_error: 0.0388 - val_loss: 0.0501 - val_mean_absolute_error: 0.0501 - 820ms/epoch - 23ms/step
Epoch 557/3000
35/35 - 1s - loss: 0.0475 - mean_absolute_error: 0.0475 - val_loss: 0.0339 - val_mean

Epoch 606/3000
35/35 - 1s - loss: 0.0368 - mean_absolute_error: 0.0368 - val_loss: 0.0348 - val_mean_absolute_error: 0.0348 - 817ms/epoch - 23ms/step
Epoch 607/3000
35/35 - 1s - loss: 0.0397 - mean_absolute_error: 0.0397 - val_loss: 0.0387 - val_mean_absolute_error: 0.0387 - 817ms/epoch - 23ms/step
Epoch 608/3000
35/35 - 1s - loss: 0.0431 - mean_absolute_error: 0.0431 - val_loss: 0.0369 - val_mean_absolute_error: 0.0369 - 779ms/epoch - 22ms/step
Epoch 609/3000
35/35 - 1s - loss: 0.0397 - mean_absolute_error: 0.0397 - val_loss: 0.0288 - val_mean_absolute_error: 0.0288 - 797ms/epoch - 23ms/step
Epoch 610/3000
35/35 - 1s - loss: 0.0419 - mean_absolute_error: 0.0419 - val_loss: 0.0369 - val_mean_absolute_error: 0.0369 - 830ms/epoch - 24ms/step
Epoch 611/3000
35/35 - 1s - loss: 0.0351 - mean_absolute_error: 0.0351 - val_loss: 0.0338 - val_mean_absolute_error: 0.0338 - 817ms/epoch - 23ms/step
Epoch 612/3000
35/35 - 1s - loss: 0.0392 - mean_absolute_error: 0.0392 - val_loss: 0.0318 - val_mean

Epoch 661/3000
35/35 - 1s - loss: 0.0392 - mean_absolute_error: 0.0392 - val_loss: 0.0461 - val_mean_absolute_error: 0.0461 - 775ms/epoch - 22ms/step
Epoch 662/3000
35/35 - 1s - loss: 0.0354 - mean_absolute_error: 0.0354 - val_loss: 0.0305 - val_mean_absolute_error: 0.0305 - 764ms/epoch - 22ms/step
Epoch 663/3000
35/35 - 1s - loss: 0.0359 - mean_absolute_error: 0.0359 - val_loss: 0.0293 - val_mean_absolute_error: 0.0293 - 798ms/epoch - 23ms/step
Epoch 664/3000
35/35 - 1s - loss: 0.0337 - mean_absolute_error: 0.0337 - val_loss: 0.0247 - val_mean_absolute_error: 0.0247 - 801ms/epoch - 23ms/step
Epoch 665/3000
35/35 - 1s - loss: 0.0351 - mean_absolute_error: 0.0351 - val_loss: 0.0310 - val_mean_absolute_error: 0.0310 - 778ms/epoch - 22ms/step
Epoch 666/3000
35/35 - 1s - loss: 0.0350 - mean_absolute_error: 0.0350 - val_loss: 0.0384 - val_mean_absolute_error: 0.0384 - 750ms/epoch - 21ms/step
Epoch 667/3000
35/35 - 1s - loss: 0.0346 - mean_absolute_error: 0.0346 - val_loss: 0.0363 - val_mean

Epoch 716/3000
35/35 - 1s - loss: 0.0358 - mean_absolute_error: 0.0358 - val_loss: 0.0272 - val_mean_absolute_error: 0.0272 - 794ms/epoch - 23ms/step
Epoch 717/3000
35/35 - 1s - loss: 0.0386 - mean_absolute_error: 0.0386 - val_loss: 0.0332 - val_mean_absolute_error: 0.0332 - 803ms/epoch - 23ms/step
Epoch 718/3000
35/35 - 1s - loss: 0.0390 - mean_absolute_error: 0.0390 - val_loss: 0.0277 - val_mean_absolute_error: 0.0277 - 721ms/epoch - 21ms/step
Epoch 719/3000
35/35 - 1s - loss: 0.0414 - mean_absolute_error: 0.0414 - val_loss: 0.0293 - val_mean_absolute_error: 0.0293 - 749ms/epoch - 21ms/step
Epoch 720/3000
35/35 - 1s - loss: 0.0353 - mean_absolute_error: 0.0353 - val_loss: 0.0371 - val_mean_absolute_error: 0.0371 - 740ms/epoch - 21ms/step
Epoch 721/3000
35/35 - 1s - loss: 0.0356 - mean_absolute_error: 0.0356 - val_loss: 0.0308 - val_mean_absolute_error: 0.0308 - 795ms/epoch - 23ms/step
Epoch 722/3000
35/35 - 1s - loss: 0.0332 - mean_absolute_error: 0.0332 - val_loss: 0.0340 - val_mean

In [16]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

11/11 [==============================] - 0s 11ms/step
0.20680109686200482


In [17]:
# pred=model.predict(X_test)
# print(mean_absolute_error(y_test,pred))
# 0.17889024475861343

In [18]:
import os
f=open('AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k[0])+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv.zip AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv'
os.system(cmd)


updating: AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv (deflated 53%)


0

# 2. SuperCon

In [19]:
from jarvis.db.figshare import  get_request_data
import pandas as pd
dat = get_request_data(js_tag="supercon_chem.json",url="https://figshare.com/ndownloader/files/40719260")
#http://supercon.nims.go.jp/index_en.html
#https://github.com/vstanev1/Supercon

df = pd.DataFrame(dat)

Loading the zipfile...
Loading completed.


In [20]:
df

,id,Tc,formula
0,1524,0.00,Tb1Th3.4Ru0.6B4
1,14309,0.00,B2Th1
2,6607,1.09,Ni2.5Cu1.5Zr8
3,4010,24.60,Pr0.91La1Ce0.09Cu1O4
4,14511,55.00,Pb0.5Ca1Sr2Y0.5Cu2O6.97
...,...,...,...
16409,442,0.00,As1Pd7
16410,14942,0.00,La1.5Gd1.5Ca1Ba2Sr1Cu6.3In0.7O16.62
16411,3978,0.00,N0.98Pr1
16412,1078,32.00,Sm0.1La1.7Sr0.2Cu1O4


In [21]:
import json,zipfile
import numpy as np
path = "../../benchmarks/AI/SinglePropertyPrediction/supercon_chem_Tc.json.zip"
js_tag = "supercon_chem_Tc.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

In [22]:
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

lgbm = lgb.LGBMRegressor(
    # device="gpu",
    n_estimators=1170,
    learning_rate=0.15,
    num_leaves=273,
)
X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['Tc'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['Tc'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')
#Split train into train-val split
n_train = int(len(X_train)*0.8)
n_val = len(X_train)-n_train
n_train,n_val,X_train.shape
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_val = X_train[-n_val:]
y_val = y_train[-n_val:]

In [23]:
X_train.shape,X_test.shape

((10504, 103), (3283, 103))

In [24]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error
#Network from "ElemNet: Deep Learning the Chemistry of Materials From Only Elemental Composition"
#Link to paper: https://www.nature.com/articles/s41598-018-35934-y
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = Activation('relu')(layer_1)

layer_2 = Dense(1024)(layer_1)
layer_2 = Activation('relu')(layer_2)

layer_3 = Dense(1024)(layer_2)
layer_3 = Activation('relu')(layer_3)

layer_4 = Dense(1024)(layer_3)
layer_4 = Activation('relu')(layer_4)
layer_4 = Dropout(0.2)(layer_4, training=True)

layer_5 = Dense(512)(layer_4)
layer_5 = Activation('relu')(layer_5)

layer_6 = Dense(512)(layer_5)
layer_6 = Activation('relu')(layer_6)

layer_7 = Dense(512)(layer_6)
layer_7 = Activation('relu')(layer_7)
layer_7 = Dropout(0.1)(layer_7, training=True)

layer_8 = Dense(256)(layer_7)
layer_8 = Activation('relu')(layer_8)

layer_9 = Dense(256)(layer_8)
layer_9 = Activation('relu')(layer_9)

layer_10 = Dense(256)(layer_9)
layer_10 = Activation('relu')(layer_10)
layer_10 = Dropout(0.3)(layer_10, training=True)

layer_11 = Dense(128)(layer_10)
layer_11 = Activation('relu')(layer_11)

layer_12 = Dense(128)(layer_11)
layer_12 = Activation('relu')(layer_12)

layer_13 = Dense(128)(layer_12)
layer_13 = Activation('relu')(layer_13)
layer_13 = Dropout(0.2)(layer_13, training=True)

layer_14 = Dense(64)(layer_13)
layer_14 = Activation('relu')(layer_14)

layer_15 = Dense(64)(layer_14)
layer_15 = Activation('relu')(layer_15)

layer_16 = Dense(32)(layer_15)
layer_16 = Activation('relu')(layer_16)

out_layer = Dense(1)(layer_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


/home/kamalch/miniconda3/envs/gpaw/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [25]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000
329/329 - 8s - loss: 13.6599 - mean_absolute_error: 13.6599 - val_loss: 12.3264 - val_mean_absolute_error: 12.3264 - 8s/epoch - 26ms/step
Epoch 2/3000
329/329 - 7s - loss: 10.3639 - mean_absolute_error: 10.3639 - val_loss: 10.0586 - val_mean_absolute_error: 10.0586 - 7s/epoch - 22ms/step
Epoch 3/3000
329/329 - 8s - loss: 9.6995 - mean_absolute_error: 9.6995 - val_loss: 9.1444 - val_mean_absolute_error: 9.1444 - 8s/epoch - 23ms/step
Epoch 4/3000
329/329 - 8s - loss: 9.1304 - mean_absolute_error: 9.1304 - val_loss: 9.1107 - val_mean_absolute_error: 9.1107 - 8s/epoch - 23ms/step
Epoch 5/3000
329/329 - 7s - loss: 8.8685 - mean_absolute_error: 8.8685 - val_loss: 9.2400 - val_mean_absolute_error: 9.2400 - 7s/epoch - 23ms/step
Epoch 6/3000
329/329 - 8s - loss: 8.5513 - mean_absolute_error: 8.5513 - val_loss: 8.6870 - val_mean_absolute_error: 8.6870 - 8s/epoch - 23ms/step
Epoch 7/3000
329/329 - 8s - loss: 8.5847 - mean_absolute_error: 8.5847 - val_loss: 8.2301 - val_mean_absolute_

Epoch 57/3000
329/329 - 7s - loss: 5.5737 - mean_absolute_error: 5.5737 - val_loss: 5.7504 - val_mean_absolute_error: 5.7504 - 7s/epoch - 22ms/step
Epoch 58/3000
329/329 - 7s - loss: 5.5469 - mean_absolute_error: 5.5469 - val_loss: 5.2840 - val_mean_absolute_error: 5.2840 - 7s/epoch - 22ms/step
Epoch 59/3000
329/329 - 7s - loss: 5.5210 - mean_absolute_error: 5.5210 - val_loss: 5.4397 - val_mean_absolute_error: 5.4397 - 7s/epoch - 22ms/step
Epoch 60/3000
329/329 - 8s - loss: 5.5549 - mean_absolute_error: 5.5549 - val_loss: 5.6329 - val_mean_absolute_error: 5.6329 - 8s/epoch - 23ms/step
Epoch 61/3000
329/329 - 7s - loss: 5.4598 - mean_absolute_error: 5.4598 - val_loss: 5.5760 - val_mean_absolute_error: 5.5760 - 7s/epoch - 22ms/step
Epoch 62/3000
329/329 - 7s - loss: 5.4983 - mean_absolute_error: 5.4983 - val_loss: 5.7405 - val_mean_absolute_error: 5.7405 - 7s/epoch - 23ms/step
Epoch 63/3000
329/329 - 7s - loss: 5.4292 - mean_absolute_error: 5.4292 - val_loss: 5.4660 - val_mean_absolute_e

Epoch 113/3000
329/329 - 7s - loss: 4.8653 - mean_absolute_error: 4.8653 - val_loss: 4.9641 - val_mean_absolute_error: 4.9641 - 7s/epoch - 22ms/step
Epoch 114/3000
329/329 - 7s - loss: 5.0721 - mean_absolute_error: 5.0721 - val_loss: 4.8224 - val_mean_absolute_error: 4.8224 - 7s/epoch - 23ms/step
Epoch 115/3000
329/329 - 7s - loss: 4.9082 - mean_absolute_error: 4.9082 - val_loss: 4.6816 - val_mean_absolute_error: 4.6816 - 7s/epoch - 22ms/step
Epoch 116/3000
329/329 - 7s - loss: 4.8404 - mean_absolute_error: 4.8404 - val_loss: 4.7155 - val_mean_absolute_error: 4.7155 - 7s/epoch - 22ms/step
Epoch 117/3000
329/329 - 7s - loss: 4.8255 - mean_absolute_error: 4.8255 - val_loss: 4.9366 - val_mean_absolute_error: 4.9366 - 7s/epoch - 21ms/step
Epoch 118/3000
329/329 - 7s - loss: 4.8196 - mean_absolute_error: 4.8196 - val_loss: 5.1175 - val_mean_absolute_error: 5.1175 - 7s/epoch - 21ms/step
Epoch 119/3000
329/329 - 7s - loss: 5.0153 - mean_absolute_error: 5.0153 - val_loss: 5.3211 - val_mean_abs

Epoch 168/3000
329/329 - 7s - loss: 4.9751 - mean_absolute_error: 4.9751 - val_loss: 4.9598 - val_mean_absolute_error: 4.9598 - 7s/epoch - 21ms/step
Epoch 169/3000
329/329 - 7s - loss: 5.0219 - mean_absolute_error: 5.0219 - val_loss: 5.0045 - val_mean_absolute_error: 5.0045 - 7s/epoch - 21ms/step
Epoch 170/3000
329/329 - 7s - loss: 4.4610 - mean_absolute_error: 4.4610 - val_loss: 4.2671 - val_mean_absolute_error: 4.2671 - 7s/epoch - 21ms/step
Epoch 171/3000
329/329 - 8s - loss: 4.4030 - mean_absolute_error: 4.4030 - val_loss: 4.3233 - val_mean_absolute_error: 4.3233 - 8s/epoch - 23ms/step
Epoch 172/3000
329/329 - 8s - loss: 4.3844 - mean_absolute_error: 4.3844 - val_loss: 4.5129 - val_mean_absolute_error: 4.5129 - 8s/epoch - 23ms/step
Epoch 173/3000
329/329 - 8s - loss: 4.3349 - mean_absolute_error: 4.3349 - val_loss: 4.3860 - val_mean_absolute_error: 4.3860 - 8s/epoch - 23ms/step
Epoch 174/3000
329/329 - 8s - loss: 4.2860 - mean_absolute_error: 4.2860 - val_loss: 4.4020 - val_mean_abs

Epoch 223/3000
329/329 - 4s - loss: 5.1513 - mean_absolute_error: 5.1513 - val_loss: 5.3622 - val_mean_absolute_error: 5.3622 - 4s/epoch - 13ms/step
Epoch 224/3000
329/329 - 4s - loss: 4.7785 - mean_absolute_error: 4.7785 - val_loss: 4.9096 - val_mean_absolute_error: 4.9096 - 4s/epoch - 13ms/step
Epoch 225/3000
329/329 - 4s - loss: 4.4044 - mean_absolute_error: 4.4044 - val_loss: 4.2907 - val_mean_absolute_error: 4.2907 - 4s/epoch - 13ms/step
Epoch 226/3000
329/329 - 4s - loss: 4.4211 - mean_absolute_error: 4.4211 - val_loss: 4.4271 - val_mean_absolute_error: 4.4271 - 4s/epoch - 13ms/step
Epoch 227/3000
329/329 - 4s - loss: 4.1291 - mean_absolute_error: 4.1291 - val_loss: 4.5966 - val_mean_absolute_error: 4.5966 - 4s/epoch - 13ms/step
Epoch 228/3000
329/329 - 4s - loss: 4.1901 - mean_absolute_error: 4.1901 - val_loss: 4.4296 - val_mean_absolute_error: 4.4296 - 4s/epoch - 13ms/step
Epoch 229/3000
329/329 - 4s - loss: 4.9662 - mean_absolute_error: 4.9662 - val_loss: 4.6231 - val_mean_abs

Epoch 278/3000
329/329 - 4s - loss: 3.9388 - mean_absolute_error: 3.9388 - val_loss: 3.8601 - val_mean_absolute_error: 3.8601 - 4s/epoch - 14ms/step
Epoch 279/3000
329/329 - 4s - loss: 3.8201 - mean_absolute_error: 3.8201 - val_loss: 3.7934 - val_mean_absolute_error: 3.7934 - 4s/epoch - 13ms/step
Epoch 280/3000
329/329 - 4s - loss: 3.8217 - mean_absolute_error: 3.8217 - val_loss: 4.2165 - val_mean_absolute_error: 4.2165 - 4s/epoch - 13ms/step
Epoch 281/3000
329/329 - 4s - loss: 3.8089 - mean_absolute_error: 3.8089 - val_loss: 3.9587 - val_mean_absolute_error: 3.9587 - 4s/epoch - 14ms/step
Epoch 282/3000
329/329 - 4s - loss: 4.2815 - mean_absolute_error: 4.2815 - val_loss: 5.0957 - val_mean_absolute_error: 5.0957 - 4s/epoch - 14ms/step
Epoch 283/3000
329/329 - 5s - loss: 4.1764 - mean_absolute_error: 4.1764 - val_loss: 4.7283 - val_mean_absolute_error: 4.7283 - 5s/epoch - 14ms/step
Epoch 284/3000
329/329 - 7s - loss: 3.8211 - mean_absolute_error: 3.8211 - val_loss: 3.7798 - val_mean_abs

Epoch 333/3000
329/329 - 4s - loss: 3.8276 - mean_absolute_error: 3.8276 - val_loss: 3.9882 - val_mean_absolute_error: 3.9882 - 4s/epoch - 13ms/step
Epoch 334/3000
329/329 - 4s - loss: 3.6887 - mean_absolute_error: 3.6887 - val_loss: 3.5593 - val_mean_absolute_error: 3.5593 - 4s/epoch - 13ms/step
Epoch 335/3000
329/329 - 4s - loss: 4.3972 - mean_absolute_error: 4.3972 - val_loss: 4.0979 - val_mean_absolute_error: 4.0979 - 4s/epoch - 13ms/step
Epoch 336/3000
329/329 - 5s - loss: 4.0569 - mean_absolute_error: 4.0569 - val_loss: 4.1166 - val_mean_absolute_error: 4.1166 - 5s/epoch - 15ms/step
Epoch 337/3000
329/329 - 9s - loss: 3.7886 - mean_absolute_error: 3.7886 - val_loss: 4.1924 - val_mean_absolute_error: 4.1924 - 9s/epoch - 27ms/step
Epoch 338/3000
329/329 - 10s - loss: 4.1030 - mean_absolute_error: 4.1030 - val_loss: 4.6025 - val_mean_absolute_error: 4.6025 - 10s/epoch - 30ms/step
Epoch 339/3000
329/329 - 11s - loss: 3.7680 - mean_absolute_error: 3.7680 - val_loss: 3.7890 - val_mean_

Epoch 388/3000
329/329 - 5s - loss: 3.3710 - mean_absolute_error: 3.3710 - val_loss: 3.6659 - val_mean_absolute_error: 3.6659 - 5s/epoch - 14ms/step
Epoch 389/3000
329/329 - 4s - loss: 3.5806 - mean_absolute_error: 3.5806 - val_loss: 4.0585 - val_mean_absolute_error: 4.0585 - 4s/epoch - 13ms/step
Epoch 390/3000
329/329 - 4s - loss: 4.9719 - mean_absolute_error: 4.9719 - val_loss: 4.0360 - val_mean_absolute_error: 4.0360 - 4s/epoch - 13ms/step
Epoch 391/3000
329/329 - 5s - loss: 3.7064 - mean_absolute_error: 3.7064 - val_loss: 3.4694 - val_mean_absolute_error: 3.4694 - 5s/epoch - 14ms/step
Epoch 392/3000
329/329 - 9s - loss: 3.8560 - mean_absolute_error: 3.8560 - val_loss: 3.9634 - val_mean_absolute_error: 3.9634 - 9s/epoch - 28ms/step
Epoch 393/3000
329/329 - 9s - loss: 4.2258 - mean_absolute_error: 4.2258 - val_loss: 3.6429 - val_mean_absolute_error: 3.6429 - 9s/epoch - 27ms/step
Epoch 394/3000
329/329 - 8s - loss: 3.7486 - mean_absolute_error: 3.7486 - val_loss: 4.0074 - val_mean_abs

Epoch 443/3000
329/329 - 9s - loss: 3.7487 - mean_absolute_error: 3.7487 - val_loss: 3.9147 - val_mean_absolute_error: 3.9147 - 9s/epoch - 26ms/step
Epoch 444/3000
329/329 - 7s - loss: 3.8891 - mean_absolute_error: 3.8891 - val_loss: 3.9353 - val_mean_absolute_error: 3.9353 - 7s/epoch - 21ms/step
Epoch 445/3000
329/329 - 4s - loss: 3.4719 - mean_absolute_error: 3.4719 - val_loss: 3.5889 - val_mean_absolute_error: 3.5889 - 4s/epoch - 13ms/step
Epoch 446/3000
329/329 - 4s - loss: 3.2763 - mean_absolute_error: 3.2763 - val_loss: 3.3731 - val_mean_absolute_error: 3.3731 - 4s/epoch - 13ms/step
Epoch 447/3000
329/329 - 4s - loss: 3.2565 - mean_absolute_error: 3.2565 - val_loss: 3.3831 - val_mean_absolute_error: 3.3831 - 4s/epoch - 13ms/step
Epoch 448/3000
329/329 - 4s - loss: 3.3824 - mean_absolute_error: 3.3824 - val_loss: 3.6426 - val_mean_absolute_error: 3.6426 - 4s/epoch - 13ms/step
Epoch 449/3000
329/329 - 4s - loss: 3.6896 - mean_absolute_error: 3.6896 - val_loss: 4.3568 - val_mean_abs

Epoch 498/3000
329/329 - 5s - loss: 3.9928 - mean_absolute_error: 3.9928 - val_loss: 4.5182 - val_mean_absolute_error: 4.5182 - 5s/epoch - 14ms/step
Epoch 499/3000
329/329 - 5s - loss: 4.0095 - mean_absolute_error: 4.0095 - val_loss: 4.0207 - val_mean_absolute_error: 4.0207 - 5s/epoch - 14ms/step
Epoch 500/3000
329/329 - 5s - loss: 3.9133 - mean_absolute_error: 3.9133 - val_loss: 4.3469 - val_mean_absolute_error: 4.3469 - 5s/epoch - 14ms/step
Epoch 501/3000
329/329 - 5s - loss: 4.0611 - mean_absolute_error: 4.0611 - val_loss: 3.9287 - val_mean_absolute_error: 3.9287 - 5s/epoch - 14ms/step
Epoch 502/3000
329/329 - 5s - loss: 3.8829 - mean_absolute_error: 3.8829 - val_loss: 4.1538 - val_mean_absolute_error: 4.1538 - 5s/epoch - 14ms/step
Epoch 503/3000
329/329 - 4s - loss: 3.9736 - mean_absolute_error: 3.9736 - val_loss: 4.1444 - val_mean_absolute_error: 4.1444 - 4s/epoch - 14ms/step
Epoch 504/3000
329/329 - 4s - loss: 4.0050 - mean_absolute_error: 4.0050 - val_loss: 4.0075 - val_mean_abs

In [26]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

103/103 [==============================] - 1s 5ms/step
5.590070764072404


In [27]:
import os
f=open('AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k[0])+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv.zip AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv'
os.system(cmd)


  adding: AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv (deflated 55%)


0

# 3. Magnetic2DChem

In [30]:
import json,zipfile
import numpy as np
from jarvis.db.figshare import  get_request_data
import pandas as pd
dat = get_request_data(js_tag="mag2d_chem.json",url="https://figshare.com/ndownloader/files/40720004")
#http://supercon.nims.go.jp/index_en.html
#https://github.com/vstanev1/Supercon

df = pd.DataFrame(dat)
path = "../../benchmarks/AI/SinglePropertyPrediction/mag2d_chem_magnetic_moment.json.zip"
js_tag = "mag2d_chem_magnetic_moment.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.64k/2.64k [00:00<00:00, 905kiB/s]

Loading the zipfile...
Loading completed.


In [31]:
df

,id,magnetic_moment,formula
0,49,5.0780,V1Cr1Ge2Te6
1,4,3.1560,Cr1Co1Si2Te6
2,212,3.0161,Ta1Cr1Sn1Ge1Te6
3,153,2.7459,Cr1Sn1Te6P1Au1
4,46,2.2800,Ti1Cr1Si2Te6
...,...,...,...
221,57,2.9670,Y1Cr1Si1Te6P1
222,97,5.6479,Cr1Ge2Te6W1
223,60,2.6981,Ta1Cr1In1Te6Pb1
224,152,5.0129,Cr1Sn1Te6Mo1P1


In [32]:
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

lgbm = lgb.LGBMRegressor(
    # device="gpu",
    n_estimators=1170,
    learning_rate=0.15,
    num_leaves=273,
)
X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['magnetic_moment'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['magnetic_moment'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')
#Split train into train-val split
n_train = int(len(X_train)*0.8)
n_val = len(X_train)-n_train
n_train,n_val,X_train.shape
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_val = X_train[-n_val:]
y_val = y_train[-n_val:]

In [33]:
X_train.shape,X_test.shape

((144, 103), (46, 103))

In [34]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error
#Network from "ElemNet: Deep Learning the Chemistry of Materials From Only Elemental Composition"
#Link to paper: https://www.nature.com/articles/s41598-018-35934-y
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = Activation('relu')(layer_1)

layer_2 = Dense(1024)(layer_1)
layer_2 = Activation('relu')(layer_2)

layer_3 = Dense(1024)(layer_2)
layer_3 = Activation('relu')(layer_3)

layer_4 = Dense(1024)(layer_3)
layer_4 = Activation('relu')(layer_4)
layer_4 = Dropout(0.2)(layer_4, training=True)

layer_5 = Dense(512)(layer_4)
layer_5 = Activation('relu')(layer_5)

layer_6 = Dense(512)(layer_5)
layer_6 = Activation('relu')(layer_6)

layer_7 = Dense(512)(layer_6)
layer_7 = Activation('relu')(layer_7)
layer_7 = Dropout(0.1)(layer_7, training=True)

layer_8 = Dense(256)(layer_7)
layer_8 = Activation('relu')(layer_8)

layer_9 = Dense(256)(layer_8)
layer_9 = Activation('relu')(layer_9)

layer_10 = Dense(256)(layer_9)
layer_10 = Activation('relu')(layer_10)
layer_10 = Dropout(0.3)(layer_10, training=True)

layer_11 = Dense(128)(layer_10)
layer_11 = Activation('relu')(layer_11)

layer_12 = Dense(128)(layer_11)
layer_12 = Activation('relu')(layer_12)

layer_13 = Dense(128)(layer_12)
layer_13 = Activation('relu')(layer_13)
layer_13 = Dropout(0.2)(layer_13, training=True)

layer_14 = Dense(64)(layer_13)
layer_14 = Activation('relu')(layer_14)

layer_15 = Dense(64)(layer_14)
layer_15 = Activation('relu')(layer_15)

layer_16 = Dense(32)(layer_15)
layer_16 = Activation('relu')(layer_16)

out_layer = Dense(1)(layer_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


/home/kamalch/miniconda3/envs/gpaw/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [35]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000
5/5 - 1s - loss: 3.3703 - mean_absolute_error: 3.3703 - val_loss: 3.2360 - val_mean_absolute_error: 3.2360 - 1s/epoch - 223ms/step
Epoch 2/3000
5/5 - 0s - loss: 3.3551 - mean_absolute_error: 3.3551 - val_loss: 3.2123 - val_mean_absolute_error: 3.2123 - 180ms/epoch - 36ms/step
Epoch 3/3000
5/5 - 0s - loss: 3.3192 - mean_absolute_error: 3.3192 - val_loss: 3.1409 - val_mean_absolute_error: 3.1409 - 313ms/epoch - 63ms/step
Epoch 4/3000
5/5 - 0s - loss: 3.2089 - mean_absolute_error: 3.2089 - val_loss: 2.9227 - val_mean_absolute_error: 2.9227 - 155ms/epoch - 31ms/step
Epoch 5/3000
5/5 - 0s - loss: 2.8585 - mean_absolute_error: 2.8585 - val_loss: 2.2390 - val_mean_absolute_error: 2.2390 - 180ms/epoch - 36ms/step
Epoch 6/3000
5/5 - 0s - loss: 1.7619 - mean_absolute_error: 1.7619 - val_loss: 1.0062 - val_mean_absolute_error: 1.0062 - 190ms/epoch - 38ms/step
Epoch 7/3000
5/5 - 0s - loss: 1.2652 - mean_absolute_error: 1.2652 - val_loss: 1.1416 - val_mean_absolute_error: 1.1416 - 163m

Epoch 57/3000
5/5 - 0s - loss: 0.5697 - mean_absolute_error: 0.5697 - val_loss: 0.4920 - val_mean_absolute_error: 0.4920 - 171ms/epoch - 34ms/step
Epoch 58/3000
5/5 - 0s - loss: 0.5938 - mean_absolute_error: 0.5938 - val_loss: 0.6276 - val_mean_absolute_error: 0.6276 - 162ms/epoch - 32ms/step
Epoch 59/3000
5/5 - 0s - loss: 0.6106 - mean_absolute_error: 0.6106 - val_loss: 0.4584 - val_mean_absolute_error: 0.4584 - 157ms/epoch - 31ms/step
Epoch 60/3000
5/5 - 0s - loss: 0.5544 - mean_absolute_error: 0.5544 - val_loss: 0.5939 - val_mean_absolute_error: 0.5939 - 151ms/epoch - 30ms/step
Epoch 61/3000
5/5 - 0s - loss: 0.6162 - mean_absolute_error: 0.6162 - val_loss: 0.4870 - val_mean_absolute_error: 0.4870 - 170ms/epoch - 34ms/step
Epoch 62/3000
5/5 - 0s - loss: 0.5748 - mean_absolute_error: 0.5748 - val_loss: 0.6435 - val_mean_absolute_error: 0.6435 - 171ms/epoch - 34ms/step
Epoch 63/3000
5/5 - 0s - loss: 0.5947 - mean_absolute_error: 0.5947 - val_loss: 0.5866 - val_mean_absolute_error: 0.58

Epoch 113/3000
5/5 - 0s - loss: 0.4039 - mean_absolute_error: 0.4039 - val_loss: 0.4330 - val_mean_absolute_error: 0.4330 - 162ms/epoch - 32ms/step
Epoch 114/3000
5/5 - 0s - loss: 0.4001 - mean_absolute_error: 0.4001 - val_loss: 0.4428 - val_mean_absolute_error: 0.4428 - 156ms/epoch - 31ms/step
Epoch 115/3000
5/5 - 0s - loss: 0.4547 - mean_absolute_error: 0.4547 - val_loss: 0.4195 - val_mean_absolute_error: 0.4195 - 151ms/epoch - 30ms/step
Epoch 116/3000
5/5 - 0s - loss: 0.4441 - mean_absolute_error: 0.4441 - val_loss: 0.4985 - val_mean_absolute_error: 0.4985 - 146ms/epoch - 29ms/step
Epoch 117/3000
5/5 - 0s - loss: 0.4163 - mean_absolute_error: 0.4163 - val_loss: 0.4830 - val_mean_absolute_error: 0.4830 - 171ms/epoch - 34ms/step
Epoch 118/3000
5/5 - 0s - loss: 0.4068 - mean_absolute_error: 0.4068 - val_loss: 0.4909 - val_mean_absolute_error: 0.4909 - 178ms/epoch - 36ms/step
Epoch 119/3000
5/5 - 0s - loss: 0.4004 - mean_absolute_error: 0.4004 - val_loss: 0.3911 - val_mean_absolute_erro

Epoch 169/3000
5/5 - 0s - loss: 0.3919 - mean_absolute_error: 0.3919 - val_loss: 0.2818 - val_mean_absolute_error: 0.2818 - 171ms/epoch - 34ms/step
Epoch 170/3000
5/5 - 0s - loss: 0.3445 - mean_absolute_error: 0.3445 - val_loss: 0.4141 - val_mean_absolute_error: 0.4141 - 148ms/epoch - 30ms/step
Epoch 171/3000
5/5 - 0s - loss: 0.4160 - mean_absolute_error: 0.4160 - val_loss: 0.5541 - val_mean_absolute_error: 0.5541 - 155ms/epoch - 31ms/step
Epoch 172/3000
5/5 - 0s - loss: 0.4107 - mean_absolute_error: 0.4107 - val_loss: 0.4343 - val_mean_absolute_error: 0.4343 - 161ms/epoch - 32ms/step
Epoch 173/3000
5/5 - 0s - loss: 0.3958 - mean_absolute_error: 0.3958 - val_loss: 0.3686 - val_mean_absolute_error: 0.3686 - 169ms/epoch - 34ms/step
Epoch 174/3000
5/5 - 0s - loss: 0.3922 - mean_absolute_error: 0.3922 - val_loss: 0.3349 - val_mean_absolute_error: 0.3349 - 172ms/epoch - 34ms/step
Epoch 175/3000
5/5 - 0s - loss: 0.3241 - mean_absolute_error: 0.3241 - val_loss: 0.3380 - val_mean_absolute_erro

Epoch 225/3000
5/5 - 0s - loss: 0.3336 - mean_absolute_error: 0.3336 - val_loss: 0.2791 - val_mean_absolute_error: 0.2791 - 169ms/epoch - 34ms/step
Epoch 226/3000
5/5 - 0s - loss: 0.3175 - mean_absolute_error: 0.3175 - val_loss: 0.2377 - val_mean_absolute_error: 0.2377 - 167ms/epoch - 33ms/step
Epoch 227/3000
5/5 - 0s - loss: 0.2849 - mean_absolute_error: 0.2849 - val_loss: 0.2562 - val_mean_absolute_error: 0.2562 - 158ms/epoch - 32ms/step
Epoch 228/3000
5/5 - 0s - loss: 0.2879 - mean_absolute_error: 0.2879 - val_loss: 0.3446 - val_mean_absolute_error: 0.3446 - 182ms/epoch - 36ms/step
Epoch 229/3000
5/5 - 0s - loss: 0.3324 - mean_absolute_error: 0.3324 - val_loss: 0.3420 - val_mean_absolute_error: 0.3420 - 177ms/epoch - 35ms/step
Epoch 230/3000
5/5 - 0s - loss: 0.3361 - mean_absolute_error: 0.3361 - val_loss: 0.3002 - val_mean_absolute_error: 0.3002 - 172ms/epoch - 34ms/step
Epoch 231/3000
5/5 - 0s - loss: 0.3368 - mean_absolute_error: 0.3368 - val_loss: 0.2967 - val_mean_absolute_erro

...

Epoch 337/3000
5/5 - 0s - loss: 0.2830 - mean_absolute_error: 0.2830 - val_loss: 0.3149 - val_mean_absolute_error: 0.3149 - 158ms/epoch - 32ms/step
Epoch 338/3000
5/5 - 0s - loss: 0.2962 - mean_absolute_error: 0.2962 - val_loss: 0.2965 - val_mean_absolute_error: 0.2965 - 172ms/epoch - 34ms/step
Epoch 339/3000
5/5 - 0s - loss: 0.2874 - mean_absolute_error: 0.2874 - val_loss: 0.2946 - val_mean_absolute_error: 0.2946 - 155ms/epoch - 31ms/step
Epoch 340/3000
5/5 - 0s - loss: 0.2685 - mean_absolute_error: 0.2685 - val_loss: 0.2587 - val_mean_absolute_error: 0.2587 - 148ms/epoch - 30ms/step
Epoch 341/3000
5/5 - 0s - loss: 0.2616 - mean_absolute_error: 0.2616 - val_loss: 0.3087 - val_mean_absolute_error: 0.3087 - 166ms/epoch - 33ms/step
Epoch 342/3000
5/5 - 0s - loss: 0.2733 - mean_absolute_error: 0.2733 - val_loss: 0.2921 - val_mean_absolute_error: 0.2921 - 174ms/epoch - 35ms/step
Epoch 343/3000
5/5 - 0s - loss: 0.2445 - mean_absolute_error: 0.2445 - val_loss: 0.3111 - val_mean_absolute_erro

Epoch 393/3000
5/5 - 0s - loss: 0.2410 - mean_absolute_error: 0.2410 - val_loss: 0.2757 - val_mean_absolute_error: 0.2757 - 156ms/epoch - 31ms/step
Epoch 394/3000
5/5 - 0s - loss: 0.2701 - mean_absolute_error: 0.2701 - val_loss: 0.2432 - val_mean_absolute_error: 0.2432 - 154ms/epoch - 31ms/step
Epoch 395/3000
5/5 - 0s - loss: 0.2531 - mean_absolute_error: 0.2531 - val_loss: 0.2211 - val_mean_absolute_error: 0.2211 - 153ms/epoch - 31ms/step
Epoch 396/3000
5/5 - 0s - loss: 0.2435 - mean_absolute_error: 0.2435 - val_loss: 0.2328 - val_mean_absolute_error: 0.2328 - 170ms/epoch - 34ms/step
Epoch 397/3000
5/5 - 0s - loss: 0.2426 - mean_absolute_error: 0.2426 - val_loss: 0.2027 - val_mean_absolute_error: 0.2027 - 189ms/epoch - 38ms/step
Epoch 398/3000
5/5 - 0s - loss: 0.2519 - mean_absolute_error: 0.2519 - val_loss: 0.2465 - val_mean_absolute_error: 0.2465 - 176ms/epoch - 35ms/step
Epoch 399/3000
5/5 - 0s - loss: 0.2657 - mean_absolute_error: 0.2657 - val_loss: 0.2570 - val_mean_absolute_erro

Epoch 449/3000
5/5 - 0s - loss: 0.2280 - mean_absolute_error: 0.2280 - val_loss: 0.2380 - val_mean_absolute_error: 0.2380 - 173ms/epoch - 35ms/step
Epoch 450/3000
5/5 - 0s - loss: 0.2383 - mean_absolute_error: 0.2383 - val_loss: 0.2323 - val_mean_absolute_error: 0.2323 - 150ms/epoch - 30ms/step
Epoch 451/3000
5/5 - 0s - loss: 0.2267 - mean_absolute_error: 0.2267 - val_loss: 0.2609 - val_mean_absolute_error: 0.2609 - 179ms/epoch - 36ms/step
Epoch 452/3000
5/5 - 0s - loss: 0.2297 - mean_absolute_error: 0.2297 - val_loss: 0.2376 - val_mean_absolute_error: 0.2376 - 171ms/epoch - 34ms/step
Epoch 453/3000
5/5 - 0s - loss: 0.2342 - mean_absolute_error: 0.2342 - val_loss: 0.2460 - val_mean_absolute_error: 0.2460 - 147ms/epoch - 29ms/step
Epoch 454/3000
5/5 - 0s - loss: 0.2350 - mean_absolute_error: 0.2350 - val_loss: 0.2456 - val_mean_absolute_error: 0.2456 - 152ms/epoch - 30ms/step
Epoch 455/3000
5/5 - 0s - loss: 0.2240 - mean_absolute_error: 0.2240 - val_loss: 0.2337 - val_mean_absolute_erro

Epoch 505/3000
5/5 - 0s - loss: 0.2231 - mean_absolute_error: 0.2231 - val_loss: 0.2351 - val_mean_absolute_error: 0.2351 - 158ms/epoch - 32ms/step
Epoch 506/3000
5/5 - 0s - loss: 0.2311 - mean_absolute_error: 0.2311 - val_loss: 0.2306 - val_mean_absolute_error: 0.2306 - 161ms/epoch - 32ms/step
Epoch 507/3000
5/5 - 0s - loss: 0.2124 - mean_absolute_error: 0.2124 - val_loss: 0.2458 - val_mean_absolute_error: 0.2458 - 161ms/epoch - 32ms/step
Epoch 508/3000
5/5 - 0s - loss: 0.2299 - mean_absolute_error: 0.2299 - val_loss: 0.1790 - val_mean_absolute_error: 0.1790 - 162ms/epoch - 32ms/step
Epoch 509/3000
5/5 - 0s - loss: 0.2211 - mean_absolute_error: 0.2211 - val_loss: 0.2262 - val_mean_absolute_error: 0.2262 - 161ms/epoch - 32ms/step
Epoch 510/3000
5/5 - 0s - loss: 0.2237 - mean_absolute_error: 0.2237 - val_loss: 0.2573 - val_mean_absolute_error: 0.2573 - 165ms/epoch - 33ms/step
Epoch 511/3000
5/5 - 0s - loss: 0.2513 - mean_absolute_error: 0.2513 - val_loss: 0.2523 - val_mean_absolute_erro

Epoch 559: early stopping
CPU times: user 6min 4s, sys: 1min 7s, total: 7min 12s
Wall time: 1min 35s


In [37]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

2/2 [==============================] - 0s 15ms/step
0.46049353388081415


In [38]:
import os
f=open('AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k[0])+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv.zip AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv'
os.system(cmd)


updating: AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv (deflated 50%)


0